<a href="https://colab.research.google.com/github/sunny0103/DeepLearning_nlp_projects/blob/main/Naver_movie_review/Naver_movie_review_BidirectioanlRNN_TorchText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install konlpy
!pip install -U torchtext==0.6.0
!pip install -U torch

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchtext
from torchtext.data import TabularDataset, Field, BucketIterator

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os

from konlpy.tag import Okt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd "/content/drive/MyDrive/Data/navermovies"

/content/drive/MyDrive/Data/navermovies


In [5]:
random_seed=42
np.random.seed(random_seed)
torch.manual_seed(random_seed)

* The following train and test dataset is results of preprocess from raw data and its process is in Naver_movie_review_cnn.ipynb

In [6]:
pd.read_csv("./train_data.csv")

,document,label
0,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
1,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1
2,걸음마 세부터 초등학교 학년생인 살용영화ㅋㅋㅋ별반개도 아까움,0
3,반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지정말 발로해도 그것보단 낫겟다...,0
4,액션이 없는데도 재미 있는 몇안되는 영화,1
...,...,...
71090,공포나 재난영화가 아니라 아예 대놓고 비급 크리쳐개그물임ㅋㅋ 음악 완전 흥겹다ㅋ 점...,0
71091,디케이드 다음에 더블 다음에 오즈인데 더블은 조금밖에 안나오네요,1
71092,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,0
71093,청춘 영화의 최고봉방황과 우울했던 날들의 자화상,1


In [7]:
pd.read_csv("./test_data.csv")

,document,label
0,뭐야 평점들은 나쁘진 않지만 짜리는 더더욱 아니잖아,0
1,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
2,아니었어도 다섯 줬을텐데 나와서 심기를 불편하게 하죠,0
3,마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가고개를 젖게한다,0
4,갈수록 개판되가는 중국영화 유치하고 내용없음 폼잡다 끝남 말도안되는 무기에 유치한남...,0
...,...,...
23697,액션영화로 기대하지말고 스릴러영화라 생각하고 보면 괜찮은 영화인듯,1
23698,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
23699,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,0
23700,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,0


In [8]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

#function for stopwords
def stopwords_process(text):
    word = [t for t in text if t not in stopwords]
    return word

In [9]:
tokenizer = Okt()

In [10]:
path = os. getcwd()

In [11]:
valid_ratio =.2

In [12]:
fix_length = 256

Text =  Field(sequential = True, # 순서반영
              tokenize = tokenizer.morphs, # tokenizer 지정
              fix_length = fix_length,  # 한 문장의 최대 길이 지정
              batch_first = True, # batch 우선
              is_target=False,
              use_vocab=True,
              tokenizer_language ='ko',
              preprocessing = stopwords_process,
              eos_token='<EOS>',
              include_lengths=False
              )

Label =  Field(sequential=False,
               batch_first= True,
               is_target=True, # target data
               dtype = torch.float32,
               use_vocab=True,
               unk_token=None
               )

In [13]:
train_data, valid_data = TabularDataset(
        path=path+'/train_data.csv',format='csv',
        fields=[('document', Text), ('label', Label)], skip_header=True).split(split_ratio=(1 - valid_ratio))

In [14]:
test_data = TabularDataset(
    path=path+'/test_data.csv', format='csv',
    fields=[('document', Text), ('label', Label)])

In [15]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [16]:
vector = torchtext.vocab.Vectors(name='./cc.ko.300.vec.gz') # fasttext korean model

In [17]:
Text.build_vocab(train_data, max_size=10000, min_freq=5, vectors=vector)
Label.build_vocab(train_data)

In [18]:
NUM_VOCABS = len(Text.vocab.stoi)
NUM_VOCABS

10003

In [19]:
BATCH_SIZE = 16
# an iterator that batches examples of similar lengths together.
train_iter = BucketIterator(dataset=train_data, batch_size=BATCH_SIZE,
                            shuffle=True, device=device,
                            sort_key=lambda x: len(x.document),
                            sort_within_batch=True)
valid_iter = BucketIterator(dataset=valid_data, batch_size=BATCH_SIZE,
                            device=device,
                            sort_key=lambda x: len(x.document),
                            sort_within_batch=True)
test_iter = BucketIterator(dataset=test_data, batch_size=BATCH_SIZE,
                           device=device,
                           sort_key=lambda x: len(x.document),
                           sort_within_batch=True)

In [20]:
class RNNClassifier(nn.Module):

  def __init__(self, input_size, word_vec_size, hidden_size, output_size, n_layers=4, dropout_rate=0.1):
    super().__init__()

    self.input_size = input_size # vocab size
    self.word_vec_size = word_vec_size
    self.hidden_size = hidden_size
    self.output_size =output_size
    self.n_layers = n_layers
    self.dropout_rate = dropout_rate

    self.embedding= nn.Embedding(input_size, word_vec_size)
    self.rnn = nn.LSTM(
        input_size=word_vec_size,
        hidden_size=hidden_size,
        num_layers=n_layers,
        dropout=dropout_rate,
        batch_first=True,
        bidirectional=True
    )
    self.classifier = nn.Linear(hidden_size*2, output_size)

  def forward(self, x):
    # x = (batch, length)
    x = self.embedding(x) # x = (bathc, length, word vec)
    x, _ = self.rnn(x) # x = (batch, length, hidden size*2)
    y = self.classifier(x[:, -1]) # y = (batch, output_size) use only last layer output

    return y

In [21]:
BATCH_SIZE = 32

class Trainer():
  def __init__(self, model, optimizer, criterion):
    self.model = model
    self.optimizer = optimizer
    self.criterion = criterion


  def train(self):
    self.model.train()
    running_loss = 0.0
    for batch in tqdm(train_iter, total=int(len(train_data) / BATCH_SIZE), desc="Training progress"):
      input = batch.document
      target = batch.label
      target = target.long()


      self.optimizer.zero_grad()
      logits = self.model(input)
      loss = self.criterion(logits, target)
      running_loss += loss.item()

      loss.backward()
      nn.utils.clip_grad_norm_(self.model.parameters(), 1)
      self.optimizer.step()
    train_loss = running_loss/len(train_data)
    if isinstance(target, torch.LongTensor) or isinstance(target, torch.cuda.LongTensor):
            accuracy = (torch.argmax(logits, dim=-1) == target).sum() / float(target.size(0))
    else:
            accuracy = 0
    tqdm.write('Train Loss : {:.3f}, Accuracy : {:.3f}'.format(train_loss, float(accuracy)))


  def validation(self):
    self.model.eval()
    running_loss = 0.0

    with torch.no_grad():
      for batch in tqdm(valid_iter, total=int(len(valid_data) / BATCH_SIZE),desc="validation progress"):
        input = batch.document
        target = batch.label
        target = target.long()

        logits = self.model(input)
        loss = self.criterion(logits, target)
        running_loss += loss.item()

      if isinstance(target, torch.LongTensor) or isinstance(target, torch.cuda.LongTensor):
              accuracy = (torch.argmax(logits, dim=-1) == target).sum() / float(target.size(0))
      else:
              accuracy = 0
      valid_loss = running_loss/len(valid_data)
      tqdm.write('Valid Loss : {:.3f}, Valid Accuracy : {:.3f}'.format(valid_loss, float(accuracy)))

In [22]:
input_size = NUM_VOCABS
word_vec_size = 100
hidden_size = 128
output_size = 2
learning_rate = 0.001
NUM_EPOCHS=3

model = RNNClassifier(input_size, word_vec_size, hidden_size, output_size, n_layers=4, dropout_rate=0.1).to(device)
optimizer = optim.Adam(model.parameters(), lr= learning_rate)
criterion = nn.CrossEntropyLoss()

In [23]:
trainer = Trainer(model, optimizer, criterion)
for epoch in range(1, NUM_EPOCHS+1):
  trainer.train()
  trainer.validation()

Training progress:   0%|          | 0/1777 [00:00<?, ?it/s]

Train Loss : 0.043, Accuracy : 0.500


validation progress:   0%|          | 0/444 [00:00<?, ?it/s]

Valid Loss : 0.043, Valid Accuracy : 0.500


Training progress:   0%|          | 0/1777 [00:00<?, ?it/s]

...

Valid Loss : 0.043, Valid Accuracy : 0.500


Training progress:   0%|          | 0/1777 [00:00<?, ?it/s]

Train Loss : 0.043, Accuracy : 0.312


validation progress:   0%|          | 0/444 [00:00<?, ?it/s]

Valid Loss : 0.043, Valid Accuracy : 0.125


In [24]:
Text.build_vocab(test_data, max_size=10000, vectors=vector)
Label.build_vocab(test_data)

In [25]:
model.eval()
with torch.no_grad():
  for batch in test_iter:
    input = batch.document
    target = batch.label
    target = target.long()

    pred = model(input)

    if isinstance(target, torch.LongTensor) or isinstance(target, torch.cuda.LongTensor):
          accuracy = (torch.argmax(pred, dim=-1) == target).sum() / float(target.size(0))
    else:
          accuracy = 0

print('Accuracy:',float(accuracy))

Accuracy: 0.4375
